In [1]:
import requests
import json
import pandas as pd
import numpy as np
import os

In [2]:
## Get Geolocations for federal states and districts
with open('C:/Users/simon/Jupyter/Covid/Dashboard/landkreise_simplify200.geojson', 'r') as read_file:
    geo_kreis = json.load(read_file)

with open('C:/Users/simon/Jupyter/Covid/Dashboard/bundeslaender_simplify200.geojson', 'r') as read_file:
    geo_land = json.load(read_file)

In [3]:
## Get population stats for federal states and districts
df_einwohner_kreis = pd.read_csv('C:/Users/simon/Jupyter/Covid/Dashboard/Einwohner_kreis.csv', encoding = 'ISO-8859-1', sep=';', dtype= {'RS': str, 'Kreis': str, 'Einwohner': int})

df_einwohner_land = pd.read_csv('C:/Users/simon/Jupyter/Covid//Dashboard/Einwohner_land.csv', encoding = 'ISO-8859-1', sep=';', dtype= {'RS': str, 'Kreis': str, 'Einwohner': int})
df_einwohner_land.head()

,Bundesland,Einwohner
0,Baden-Württemberg,11100394
1,Bayern,13124737
2,Berlin,3669491
3,Brandenburg,2521893
4,Bremen,681202


In [4]:
## Get Covid stats from RKI
covid_url = 'https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.geojson'
r = requests.get(covid_url)
r.status_code

200

In [5]:
a = json.loads(r.content)
df = pd.json_normalize(a['features'])

df.head()

,type,geometry,properties.IdBundesland,properties.Bundesland,properties.Landkreis,properties.Altersgruppe,properties.Geschlecht,properties.AnzahlFall,properties.AnzahlTodesfall,properties.ObjectId,properties.Meldedatum,properties.IdLandkreis,properties.Datenstand,properties.NeuerFall,properties.NeuerTodesfall,properties.Refdatum,properties.NeuGenesen,properties.AnzahlGenesen,properties.IstErkrankungsbeginn,properties.Altersgruppe2
0,Feature,None,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,1,2020-09-30T00:00:00Z,01001,"18.09.2021, 00:00 Uhr",0,-9,2020-09-30T00:00:00Z,0,1,0,Nicht übermittelt
1,Feature,None,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2,2020-10-29T00:00:00Z,01001,"18.09.2021, 00:00 Uhr",0,-9,2020-10-29T00:00:00Z,0,1,0,Nicht übermittelt
2,Feature,None,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,3,2020-11-03T00:00:00Z,01001,"18.09.2021, 00:00 Uhr",0,-9,2020-11-03T00:00:00Z,0,1,0,Nicht übermittelt
3,Feature,None,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,4,2020-11-20T00:00:00Z,01001,"18.09.2021, 00:00 Uhr",0,-9,2020-11-19T00:00:00Z,0,1,1,Nicht übermittelt
4,Feature,None,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,5,2020-11-23T00:00:00Z,01001,"18.09.2021, 00:00 Uhr",0,-9,2020-11-18T00:00:00Z,0,1,1,Nicht übermittelt


In [6]:
## Clean-up and data preparation
df.columns = df.columns.str.replace('properties.', '')
df = df[['IdBundesland', 'Bundesland', 'Landkreis', 'IdLandkreis', 'AnzahlFall', 'Altersgruppe', 'AnzahlTodesfall', 'Geschlecht', 'Meldedatum']]
df['Meldedatum'] = df['Meldedatum'].str.replace('T00:00:00Z', '')
df['Meldedatum'] = pd.to_datetime(df['Meldedatum'])

df = df.merge(df_einwohner_kreis, how='left', left_on='IdLandkreis', right_on='RS')
df = df.merge(df_einwohner_land, how='left', left_on='Bundesland', right_on='Bundesland') 

df_gender = pd.get_dummies(df['Geschlecht']).mul(df['AnzahlFall'],0)
df_gender.columns = ['Sex_M', 'Sex_F', 'Sex_Unbekannt']
df = df.join(df_gender)

df_age = pd.get_dummies(df['Altersgruppe']).mul(df['AnzahlFall'],0)
df = df.join(df_age)



df.drop(['RS', 'Kreis'], axis=1, inplace=True)

df.head()

<ipython-input-6-157b8c82909c>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('properties.', '')


,IdBundesland,Bundesland,Landkreis,IdLandkreis,AnzahlFall,Altersgruppe,AnzahlTodesfall,Geschlecht,Meldedatum,Einwohner_x,...,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,A60-A79,A80+,unbekannt
0,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-09-30,90164.0,...,1,0,0,1,0,0,0,0,0,0
1,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-10-29,90164.0,...,1,0,0,1,0,0,0,0,0,0
2,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-11-03,90164.0,...,1,0,0,1,0,0,0,0,0,0
3,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-11-20,90164.0,...,1,0,0,1,0,0,0,0,0,0
4,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-11-23,90164.0,...,1,0,0,1,0,0,0,0,0,0


In [7]:
df_bund = df.groupby(by=['Meldedatum']).agg({
    'AnzahlFall':'sum',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})


#df_bund.set_index('Region', append=True, inplace=True)
#df_bund = df_bund.swaplevel('Region', 'Meldedatum')

df_bund['Fall_Last7'] = df_bund['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_bund['Fall_MA7'] = df_bund['AnzahlFall'].rolling(7).mean()
df_bund['R7'] = (df_bund['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_bund['Einwohner'] = df_einwohner_land['Einwohner'].sum()
df_bund['Fall_100K'] = ((df_bund['Fall_Last7'] / df_bund['Einwohner']) * 100000).round(2)

df_bund.reset_index(inplace=True)
df_bund['Region'] = 'Deutschland'
df_bund['Typ'] = 'Nation'


df_bund.tail()

,Meldedatum,AnzahlFall,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,A60-A79,A80+,unbekannt,Fall_Last7,Fall_MA7,R7,Einwohner,Fall_100K,Region,Typ
581,2021-09-13,6794,10,3372,3321,101,290,1387,2301,2129,458,222,7,71318.0,10188.285714,0.96,83166711,85.75,Deutschland,Nation
582,2021-09-14,13270,9,6585,6537,148,573,2861,4428,4187,846,369,6,69483.0,9926.142857,0.95,83166711,83.55,Deutschland,Nation
583,2021-09-15,11990,12,6015,5788,187,547,2587,4033,3695,795,324,9,67343.0,9620.428571,0.93,83166711,80.97,Deutschland,Nation
584,2021-09-16,9538,4,4798,4585,155,464,2141,3197,2860,629,239,8,64538.0,9219.714286,0.90,83166711,77.60,Deutschland,Nation
585,2021-09-17,6324,2,3099,3092,133,314,1355,2122,1841,471,181,40,59754.0,8536.285714,0.84,83166711,71.85,Deutschland,Nation


In [8]:
df_land = df.groupby(by=['Bundesland', 'Meldedatum']).agg({
    'IdBundesland': 'first',
    'AnzahlFall': 'sum',
    'Einwohner_y': 'first',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})

df_land.rename(columns={'Einwohner_y': 'Einwohner'}, inplace=True)

#df_land.index.names = ['Region', 'Meldedatum']

df_land['Fall_MA4'] = df_land.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(4).mean().values
df_land['Fall_MA7'] = df_land.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(7).mean().values

df_land['R4'] = df_land['Fall_MA4'].pct_change(periods=4) + 1
df_land['R7'] = (df_land['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_land['Fall_Last7'] = df_land['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_land['Fall_100K'] = ((df_land['Fall_Last7'] / df_land['Einwohner']) * 100000).round(2)

#

df_land.reset_index(inplace=True)
df_land['Region'] = df_land['Bundesland']
df_land['Typ'] = 'Bundesland'

df_land.tail()

,Bundesland,Meldedatum,IdBundesland,AnzahlFall,Einwohner,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,...,A80+,unbekannt,Fall_MA4,Fall_MA7,R4,R7,Fall_Last7,Fall_100K,Region,Typ
8914,Thüringen,2021-09-13,16,153,2133378,0,73,80,0,11,...,7,0,111.00,149.714286,0.616667,1.06,1048.0,49.12,Thüringen,Bundesland
8915,Thüringen,2021-09-14,16,256,2133378,0,125,131,0,12,...,4,0,134.50,152.857143,0.702350,1.07,1070.0,50.16,Thüringen,Bundesland
8916,Thüringen,2021-09-15,16,193,2133378,0,98,92,3,12,...,5,0,159.00,151.285714,1.014354,1.04,1059.0,49.64,Thüringen,Bundesland
8917,Thüringen,2021-09-16,16,228,2133378,0,130,96,2,13,...,4,0,207.50,160.142857,1.816193,1.11,1121.0,52.55,Thüringen,Bundesland
8918,Thüringen,2021-09-17,16,134,2133378,0,71,63,0,8,...,2,0,202.75,156.142857,1.826577,1.04,1093.0,51.23,Thüringen,Bundesland


In [9]:
df_kreis = df.groupby(by=['Landkreis', 'Meldedatum']).agg({
    'IdLandkreis': 'first',
    'Bundesland': 'first',
    'AnzahlFall': 'sum',
    'Einwohner_x': 'first',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})
    
df_kreis.rename(columns={'Einwohner_x': 'Einwohner'}, inplace=True)


#df_kreis.index.names = ['Region', 'Meldedatum']

df_kreis['Fall_MA4'] = df_kreis.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(4).mean().values
df_kreis['Fall_MA7'] = df_kreis.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(7).mean().values

df_kreis['R4'] = df_kreis['Fall_MA4'].pct_change(periods=4) + 1
df_kreis['R7'] = (df_kreis['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_kreis['Fall_Last7'] = df_kreis['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_kreis['Fall_100K'] = ((df_kreis['Fall_Last7'] / df_kreis['Einwohner']) * 100000).round(2)

df_kreis.reset_index(inplace=True)
df_kreis['Region'] = df_kreis['Landkreis']
df_kreis['Typ'] = 'Landkreis'

df_kreis.tail()

,Landkreis,Meldedatum,IdLandkreis,Bundesland,AnzahlFall,Einwohner,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,...,A80+,unbekannt,Fall_MA4,Fall_MA7,R4,R7,Fall_Last7,Fall_100K,Region,Typ
175075,Städteregion Aachen,2021-09-13,05334,Nordrhein-Westfalen,57,557026.0,0,25,32,0,...,8,0,42.00,59.285714,0.528302,0.94,415.0,74.50,Städteregion Aachen,Landkreis
175076,Städteregion Aachen,2021-09-14,05334,Nordrhein-Westfalen,86,557026.0,0,40,46,0,...,1,0,45.25,61.285714,0.565625,0.98,429.0,77.02,Städteregion Aachen,Landkreis
175077,Städteregion Aachen,2021-09-15,05334,Nordrhein-Westfalen,55,557026.0,0,30,25,0,...,2,0,50.00,56.142857,0.704225,0.90,393.0,70.55,Städteregion Aachen,Landkreis
175078,Städteregion Aachen,2021-09-16,05334,Nordrhein-Westfalen,57,557026.0,0,31,26,0,...,1,0,63.75,52.285714,1.307692,0.85,366.0,65.71,Städteregion Aachen,Landkreis
175079,Städteregion Aachen,2021-09-17,05334,Nordrhein-Westfalen,15,557026.0,0,6,9,0,...,2,0,53.25,44.000000,1.267857,0.74,308.0,55.29,Städteregion Aachen,Landkreis


In [10]:
## Final dataset
df_merged = pd.concat([df_bund, df_land, df_kreis])

## The geolocations for the districts of Berlin are not included in the dataset, 
## to have Berlin displayed on the choropleth map, I substitute the values for the federal state of Berlin.
df_merged.loc[df_merged['Bundesland'] == 'Berlin', 'IdLandkreis'] = '11000'

df_merged.tail()

,Meldedatum,AnzahlFall,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,...,Einwohner,Fall_100K,Region,Typ,Bundesland,IdBundesland,Fall_MA4,R4,Landkreis,IdLandkreis
175075,2021-09-13,57,0,25,32,0,5,10,17,12,...,557026.0,74.50,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,42.00,0.528302,Städteregion Aachen,05334
175076,2021-09-14,86,0,40,46,0,4,19,29,29,...,557026.0,77.02,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,45.25,0.565625,Städteregion Aachen,05334
175077,2021-09-15,55,0,30,25,0,3,11,18,17,...,557026.0,70.55,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,50.00,0.704225,Städteregion Aachen,05334
175078,2021-09-16,57,0,31,26,0,1,13,16,21,...,557026.0,65.71,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,63.75,1.307692,Städteregion Aachen,05334
175079,2021-09-17,15,0,6,9,0,0,1,7,3,...,557026.0,55.29,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,53.25,1.267857,Städteregion Aachen,05334


In [11]:
if os.path.exists('covid_panel_backup.csv'):
    os.remove('covid_panel_backup.csv')

In [12]:
os.rename('covid_panel.csv', 'covid_panel_backup.csv')

In [13]:
df_merged.to_csv('covid_panel.csv')